In [1]:
from kafka import KafkaConsumer
from kafka import KafkaProducer
import json
import numpy as np
import re
from datetime import datetime
import nltk
import pickle
from nltk.corpus import stopwords
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.stem import WordNetLemmatizer

In [2]:
consumer = KafkaConsumer(
    bootstrap_servers='kafka-cluster-kafka-bootstrap.kafka-cluster-project.svc.cluster.local:9092',
    auto_offset_reset='latest',
    value_deserializer=lambda m: json.loads(m)
)
consumer.subscribe('data_to_predict_cnn')

In [ ]:
# pip install tensorflow


In [3]:
import tensorflow as tf 
load_model = tf.keras.models.load_model('cnn_model_without_duration.h5')
print(load_model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 36, 36, 64)        640       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 6, 6, 256)         295168    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 3, 3, 256)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)             

In [4]:
from kafka import KafkaConsumer
from kafka import KafkaProducer

In [5]:
#Use when using KafkaProducer to produce decision

import json
from json import JSONEncoder
import numpy

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

In [6]:
producer2 = KafkaProducer(bootstrap_servers='kafka-cluster-kafka-bootstrap.kafka-cluster-project.svc.cluster.local:9092',value_serializer=lambda v: json.dumps(v, cls=NumpyArrayEncoder).encode('utf-8'))


In [ ]:
# conda install python=3.7.6

In [ ]:
# import json

# for message in consumer:
#     #print(message)
# #     movieId = message.value['movieId']
# #     userId = message.value['userId']
# #     print("movieId is "+str(movieId))
# #     print("userId is "+str(userId))
#     finalNumpyArray = np.asarray(message.value["array"])
#     result1=load_model.predict(finalNumpyArray)
# #     final_comment = {
# #     "schema": {
# #         "type": "struct",
# #         "fields": [{
# #             "type": "string",
# #             "optional": True,
# #             "field": "spanId"
# #         }, {
# #             "type": "int64",
# #             "optional": True,
# #             "field": "movieId"
# #         }, {
# #             "type": "int64",
# #             "optional": False,
# #             "field": "userId"
# #         }, {
# #             "type": "string",
# #             "optional": False,
# #             "field": "comment"
# #         }],
# #         "optional": False,
# #         "name": "comment_schema"
# #     },
# #     "payload": {
# #         "spanId": message.value['spanId'],
# #         "movieId": message.value['movieId'],
# #         "userId": message.value['userId'],
# #         "comment": message.value["comment"]
# #     }
# # }
#     final_comment = {
#         "spanId":message.value['spanId'],
#         "movieId":message.value['movieId'],
#         "userId":message.value['userId'],
#         "comment":message.value["comment"]
#     }
#     if result1>0.5:
#         final_comment['status'] = 'anomolous'
#         #final_comment['payload']['status'] = 'anomolous'
#         #producer.send('predicted_anomolous',final_comment)
#         print("ALERT :::: This can be SQL injection")
#     elif result1<=0.5:
#         final_comment['status'] = 'normal'
#         #final_comment['payload']['status'] = 'normal'
#         #producer.send('predicted_normal',final_comment)
#         #producer.produce(topic='predicted_normal', value=final_comment, on_delivery=acked)
#         #producer.produce(topic='test_predict',value=final_comment,on_delivery=delivery_report)
#         #print(producer)
#         print("It seems to be safe")
#     while True:
#         producer.poll(0.0)
#     # Serve on_delivery callbacks from previous calls to produce()
     
#         try:
#             new_comment = comment_data(spanId=final_comment['spanId'],
#                         movieId=final_comment['movieId'],
#                         userId=final_comment['userId'],
#                         comment=final_comment['comment'],
#                         status = final_comment['status'])
#             producer.produce(topic='test_predict', value=new_comment,
#                              on_delivery=delivery_report)
# #         user_name = input("Enter name: ")
# #         user_address = input("Enter address: ")
# #         user_favorite_number = int(input("Enter favorite number: "))
# #         user_favorite_color = input("Enter favorite color: ")
        
          
#         except KeyboardInterrupt:
#             break
    
#         except ValueError:
#             print("Invalid input, discarding record...")
#             continue

#     print("\nFlushing records...")
#     producer.flush()
        
#     print(new_comment)

In [ ]:
import json
import datetime

for message in consumer:
    #print(message)
#     movieId = message.value['movieId']
#     userId = message.value['userId']
#     print("movieId is "+str(movieId))
#     print("userId is "+str(userId))
    finalNumpyArray = np.asarray(message.value["array"])
    result1=load_model.predict(finalNumpyArray)
    final_comment = {

    "schema": {
        "type": "struct",
        "optional": False,
        "version": 1,
        "fields": [
            {
                "field": "spanid",
                "type": "string",
                "optional": True
            },
            {
                "field": "movieid",
                "type": "string",
                "optional": True
            },
            {
                "field": "userid",
                "type": "string",
                "optional": True
            },
            {
                "field": "comment",
                "type": "string",
                "optional": True
            },
            {
                "field": "status",
                "type": "string",
                "optional": True
            },
            {
                "field": "comment_date",
                "type": "string",
                "optional": True
            }
        ]
    },
    "payload": {
        "spanid": message.value['spanId'],
        "movieid": message.value['movieId'],
        "userid": message.value['userId'],
        "comment": message.value["comment"],
        "comment_date": message.value["comment_date"]   
    }

    }
    if result1>0.5:
#         comment_date = datetime.date.today()
#         final_comment['payload']['comment_date'] = comment_date
        final_comment['payload']['status'] = 'anomolous'
        #final_comment['payload']['status'] = 'anomolous'
        producer2.send('predicted_normal_1',final_comment)
        print("ALERT :::: This can be SQL injection")
    elif result1<=0.5:
#         comment_date = datetime.date.today()
#         final_comment['payload']['comment_date'] = comment_date
        final_comment['payload']['status'] = 'normal'
        #final_comment['payload']['status'] = 'normal'
        producer2.send('predicted_normal_1',final_comment)
        #producer.produce(topic='predicted_normal', value=final_comment, on_delivery=acked)
        #producer.produce(topic='test_predict',value=final_comment,on_delivery=delivery_report)
        #print(producer)
        print("It seems to be safe")

#     producer.poll(0.0)
#     # Serve on_delivery callbacks from previous calls to produce()
#     try:
#         new_comment = comment_data(spanId=final_comment['spanId'],
#                         movieId=final_comment['movieId'],
#                         userId=final_comment['userId'],
#                         comment=final_comment['comment'],
#                         status = final_comment['status'])
#         producer.produce(topic='test_predict', key=str(uuid4()),value=new_comment,
#                              on_delivery=delivery_report)
# #         user_name = input("Enter name: ")
# #         user_address = input("Enter address: ")
# #         user_favorite_number = int(input("Enter favorite number: "))
# #         user_favorite_color = input("Enter favorite color: ")
        
          
#     except KeyboardInterrupt:
#         break
    
#     except ValueError:
#         print("Invalid input, discarding record...")
#         continue

#     print("\nFlushing records...")
#     producer.flush()
        
    print(final_comment)

It seems to be safe
{'schema': {'type': 'struct', 'optional': False, 'version': 1, 'fields': [{'field': 'span_id', 'type': 'string', 'optional': True}, {'field': 'movie_id', 'type': 'string', 'optional': True}, {'field': 'user_id', 'type': 'string', 'optional': True}, {'field': 'comment', 'type': 'string', 'optional': True}, {'field': 'status', 'type': 'string', 'optional': True}, {'field': 'comment_date', 'type': 'string', 'optional': True}]}, 'payload': {'span_id': 'kviwS+9Bi20=', 'movie_id': '2', 'user_id': '0', 'comment': 'hello', 'comment_date': '2021-10-13', 'status': 'normal'}}
It seems to be safe
{'schema': {'type': 'struct', 'optional': False, 'version': 1, 'fields': [{'field': 'span_id', 'type': 'string', 'optional': True}, {'field': 'movie_id', 'type': 'string', 'optional': True}, {'field': 'user_id', 'type': 'string', 'optional': True}, {'field': 'comment', 'type': 'string', 'optional': True}, {'field': 'status', 'type': 'string', 'optional': True}, {'field': 'comment_date'